In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

First we'll open the various docs and see how big they are.

In [2]:
df = pd.read_csv('../data/training_set_values.csv')

In [3]:
df.shape

(59400, 40)

In [4]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [5]:
df = pd.read_csv('../data/training_set_labels.csv')

In [6]:
df.shape

(59400, 2)

In [7]:
df.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [8]:
df.status_group.value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [9]:
df = pd.read_csv('../data/test_set_values.csv')

In [10]:
df.shape

(14850, 40)

In [11]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [12]:
df = pd.read_csv('../data/SubmissionFormat.csv')

In [13]:
df.shape

(14850, 2)

In [14]:
df.head()

,id,status_group
0,50785,predicted label
1,51630,predicted label
2,17168,predicted label
3,45559,predicted label
4,49871,predicted label


# Summary of available files
There is a training set consisting of 59,400 records, a set of labels for those 59,400 records,  a test set consisting of 14,850 records, and a template for submitting labels for those 14,850 test records to an online contest.

Because we do not have labels for the 14,850 records in the test set, those are useless to us. We will have to carve out training and test sets from the 59,400 records with known labels.

# Examining the training set

Let's look closer at the training set. First we'll drop duplicates.

In [15]:
# reload the training set
df = pd.read_csv('../data/training_set_values.csv')
# check for duplicates, excluding the ids
df[df.duplicated(subset=df.columns.difference(['id']))].shape

(37, 40)

In [16]:
# drop the duplicate records
df.drop(df[df.duplicated(subset=df.columns.difference(['id']))].index, inplace=True)

Now we'll look at missing values.

In [17]:
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3314
recorded_by                  0
scheme_management         3877
scheme_name              28138
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

"Scheme name" stands out as particularly bad. We'll surely drop this column, but let's take a look at it first.

In [18]:
df['scheme_name'].value_counts()

K                         682
None                      644
Borehole                  545
Chalinze wate             405
M                         400
                         ... 
Lake Tanganyika Basin       1
BL Embokoi                  1
Sola                        1
water supply in Mwanza      1
Dindimu Water Supply        1
Name: scheme_name, Length: 2696, dtype: int64

It's not apparent how this feature could be salvaged into anything useful.

In [19]:
df.drop(columns=['scheme_name'], inplace=True)

For the other 6 columns with missing values, let's look at how many unique values each has.

In [31]:
df[['funder', 'installer', 'subvillage', 'public_meeting', 'scheme_management', 'permit']].nunique()

funder                1897
installer             2145
subvillage           19287
public_meeting           2
scheme_management       12
permit                   2
dtype: int64

In [33]:
df[['public_meeting', 'permit']].value_counts()

public_meeting  permit
True            True      34732
                False     13845
False           False      2379
                True       2308
dtype: int64

Let's look at just numerical features.

In [ ]:
df.select_dtypes(include=['number']).info()

None appear to be missing values, but this can be deceiving. Let's look closer for zeros.

In [ ]:
df.select_dtypes(include=['int','float']).describe()

In [ ]:
df.amount_tsh.value_counts()

In [ ]:
df.gps_height.value_counts()

In [ ]:
df.longitude.value_counts()

In [ ]:
df.latitude.value_counts()

In [ ]:
df.num_private.value_counts()

In [ ]:
df.region_code.value_counts()

In [ ]:
df.district_code.value_counts()

In [ ]:
df.population.value_counts()

In [ ]:
df.construction_year.value_counts()

# A first-glance summary of the numerical features

None of the numerical features are missing any values, but some have a suspicious amount of zeros.

* amount_tsh: "Total Static Head". Some kind of measure of available water. This is mostly zeroes.
* gps_height: From a spot check, this appears to be given in meters.
* longitude, latitude: 1,812 records are essentially (0,0).
* num_private: It is not at all clear what this means.
* region_code, district_code: There are fewer unique district codes, suggesting they are possibly broader? There are some district codes equal to zero, which may or may not be an error.
* population: There are many zeroes, but perhaps this means they're just rural?
* construction_year: There are many zeroes, which will need to be dealt with.

First we'll check for duplicate records, excluding the ids, and drop any duplicates we find.

Longitude and latitude provide a helpful visual element, allowing us to create geographical color-coded maps of the other features. Let's first look at the 1,812 records that lack positional coordinates and see whether they're worth repairing.

In [ ]:
df.select_dtypes(include=['number'])[df['longitude'] == 0].describe()

These records (now 1,776 of them after dropping duplicates) seem worthless. The only numerical data they offer are their region and district codes. Let's find out where those regions and districts are before we drop the records.

In [ ]:
df[df['longitude'] == 0]['region_code'].value_counts()

In [ ]:
df[df['longitude'] == 0]['district_code'].value_counts()

Maybe if we plot the positions of records *with* GPS coordinates from these districts, we'll get some idea of where these problematic records are coming from.

In [ ]:
# generate a geographical map of all listings in the districts where the 1,776 records lacking long/lat are
fig, ax = plt.subplots(figsize=(11,8))
df[(df['longitude'] != 0) & (df['district_code'].isin([1,2,4,6]))].plot.scatter(
    x='longitude', y='latitude', c='district_code', cmap='Blues', ax=ax)
fig.suptitle('Distribution of Districts with Missing GPS Coordinates', size=18);

While the 1,776 problematic records all come from the same handful of districts, the map offers no other clues about what they may have in common, because it appears that "districts" and "regions" are not essentially contiguous but rather each consist of several discrete clusters. In any case, it looks like it won't be possible to recover or even approximate the GPS data for these records, so we might as well drop them.

In [ ]:
df.drop(df[df['longitude'] == 0].index, inplace=True)

Next, let's make a color-coded plot to see whether the gps height data makes sense by comparing it to an available topographical map.

In [ ]:
# set figure with two axes over two columns
fig, (ax1, ax2) = plt.subplots(ncols = 2, figsize=(20,8))
# plot long/lat for nonzero longs with color gradient for elevation
df.plot.scatter(x='longitude', y='latitude', c='gps_height', cmap='plasma', ax=ax1)
# upload an image
im = plt.imread("../images/topo_map.jpeg")
# display the image
im = ax2.imshow(im)
# hide X and Y axes label marks
ax2.xaxis.set_tick_params(labelbottom=False)
ax2.yaxis.set_tick_params(labelleft=False)
# hide X and Y axes tick marks
ax2.set_xticks([])
ax2.set_yticks([])
# title
fig.suptitle('Topographical Map Comparison', size=18)
fig.tight_layout();

It makes sense for the locations along the ocean to have zero elevation, but there are at least three inland clusters that seem more like they are just lacking elevation data. One solution would be to set elevation values equal to the median for all records that have matching geographical location features such as "subvillage", but it appears that this wouldn't help because the missing values are geographically set apart from non-missing values.

# *what to do about this?*

In [ ]:
df.select_dtypes(include=['object']).nunique()

In [ ]:
df.select_dtypes(include=['object']).isna().sum()

In [ ]:
df[df['installer'].isna()].isna().sum()

# A first-glance summary of  non-numerical features

There are 6 features with missing values:

* funder (3,569)
* installer (3,636)
* public_meeting (203)
* scheme_management (54)
* scheme_name (1,785)
* permit (2,413)

